<a href="https://colab.research.google.com/github/DorotaJanosz/machine-learning-bootcamp/blob/master/unsupervised/03_association_rules/02_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import bibliotek

In [11]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

###Załadowanie danych

In [12]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2020-07-09 13:52:54--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.128, 172.217.193.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv.6’

products.csv.6      100%[===================>]   2.07M  --.-KB/s    in 0.01s   

2020-07-09 13:52:54 (146 MB/s) - ‘products.csv.6’ saved [2166953/2166953]

--2020-07-09 13:52:56--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet

In [13]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name']) 
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [14]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


###Przygotowanie danych

In [15]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True) 
data = data.sort_values(by='order_id') 
data = data[:10000]
data

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado
...,...,...,...
1274070,23978,46388,Sweet & Salty Nut Cashews Granola
284158,23978,11421,Moroccan Mint Herbal Tea
317183,23978,12785,Deluxe Original Cheddar Macaroni & Cheese Dinner
128449,23993,5450,Small Hass Avocado


In [16]:
data.describe()


,order_id,product_id
count,10000.00,10000.00
mean,12231.02,25645.69
std,6828.94,14164.58
min,1.00,1.00
25%,5985.00,13176.00
50%,12525.00,25480.50
75%,18172.75,38217.50
max,23993.00,49683.00


In [17]:
#rozkład produktów 

data['product_name'].value_counts()

Banana                                                        123
Organic Strawberries                                           98
Bag of Organic Bananas                                         94
Organic Baby Spinach                                           70
Large Lemon                                                    66
                                                             ... 
Whole Medjool Dates                                             1
Slim Cut Swiss Reduced Fat Made with 2% Milk Cheese Slices      1
Microwave-Ready Idaho Potato                                    1
Pesto Pizza                                                     1
Sriracha Hot Sauce                                              1
Name: product_name, Length: 4660, dtype: int64

In [18]:
#liczba transakcji
data['order_id'].nunique() 

939

In [19]:
transactions = data.groupby(by='order_id')['product_name']


In [20]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))

transactions


order_id
1        Organic Whole String Cheese,Organic Celery Hea...
36       Super Greens Salad,Spring Water,Organic Garnet...
38       Green Peas,Organic Baby Arugula,Shelled Pistac...
96       Organic Raspberries,Organic Pomegranate Kernel...
98       Uncured Applewood Smoked Bacon,Black Beans,Org...
                               ...                        
23886    Raspberries,Organic Strawberries,Organic Blueb...
23887    Honey Maid Graham Crackers,Soft & Smooth100% W...
23960    The Smurfs Gummi Candy,Ricotta Whole Milk Chee...
23978    Rich Taste Extra Virgin Olive Oil,Original Pot...
23993            Small Hass Avocado,Organic Grape Tomatoes
Name: product_name, Length: 939, dtype: object

In [21]:
transactions = transactions.str.split(',')
transactions

order_id
1        [Organic Whole String Cheese, Organic Celery H...
36       [Super Greens Salad, Spring Water, Organic Gar...
38       [Green Peas, Organic Baby Arugula, Shelled Pis...
96       [Organic Raspberries, Organic Pomegranate Kern...
98       [Uncured Applewood Smoked Bacon, Black Beans, ...
                               ...                        
23886    [Raspberries, Organic Strawberries, Organic Bl...
23887    [Honey Maid Graham Crackers, Soft & Smooth100%...
23960    [The Smurfs Gummi Candy, Ricotta Whole Milk Ch...
23978    [Rich Taste Extra Virgin Olive Oil, Original P...
23993         [Small Hass Avocado, Organic Grape Tomatoes]
Name: product_name, Length: 939, dtype: object

###Kodowanie transakcji

In [22]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoded = encoder.transform(transactions, sparse=True) 
transactions_encoded

<939x4848 sparse matrix of type '<class 'numpy.bool_'>'
	with 10385 stored elements in Compressed Sparse Row format>

In [23]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df

,Apricot & Banana Stage 2 Baby Food,& Baby Wipes,& Cheese Biscuit,& Cheese Sandwiches,1000 Sheet Rolls,12 Ct,18 Grains and Seeds,1‑Ply,2% Milkfat,2011,33% Cocoa,50 Loads,50% Less Sugar & Calories,8 Regular Rolls Towels/Napkins,97% Fat Free,Aged Parmesan and Romano Cheeses Mixed Cheese Tortellini,Aged Parmesan and Romano Cheeses Three Cheese Tortellini,Aged White Cheddar,All-One,Almond Milk,Amalfi Lemon,Americano,Apple,Ayurvedic Herbal,Baby Bok Choy,Bacon Habanero,Bag,Bags,Barrel-Aged,Basil & Garlic,Bibb) Lettuce,Bite Size,Black Olives) Frozen Pizza,Blackberry,Blood Orange,Blueberries & Beets + Super Chia Fruit & Veggie Blend,Blueberries & Spinach Baby Food,Blueberry,Bonus Bag,Botanical,...,Yakisoba Teriyaki Beef Japanese Noodles,Yellow Bell Pepper,Yellow Corn Organic Tortillas,Yellow Grape Tomatoes,Yellow Onion,Yellow Onions,Yellow Straightneck Squash,Yellow Tortilla Chips,Yo Baby Organic Whole Milk Banana Mango Yogurt,Yo Toddler Organic Strawberry Banana Whole Milk Yogurt,YoBaby Blueberry Apple Yogurt,YoBaby Peach Pear Yogurt,YoKids Squeeze! Organic Strawberry Flavor Yogurt,YoKids Squeezers Organic Low-Fat Yogurt,YoKids Strawberry Banana/Strawberry Yogurt,Yobaby Organic Plain Yogurt,Yoghurt Blueberry,Yogurt,Yogurt Tubes Blueberry,Yokids Organic Strawberry Banana Lowfat Yogurt Squeeze,Yotoddler Organic Pear Spinach Mango Yogurt,Yukon Gold Potatoes,Yukon Gold Potatoes 5lb Bag,Z Bar Protein Peanut Butter Chocolate Protein Snack Bar,ZBar Organic Chocolate Brownie Energy Snack,Zebra Cake Rolls,Zero Calorie Cola,Zero Calorie Cream Soda,Zero Calorie Orange Soda,Zero Calorie Sweetener,Zero Coke Soda Mini Bottles,Zero Cola,Zero Go-Go Mixed Berry Vitamin Water,Zero Soda,Zero Strawberry Lemon,Zucchini Squash,of Norwich Original English Mustard Powder Double Superfine,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

###Algorytm Apriori

In [24]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True, n_jobs=-1 )
supports = supports.sort_values(by='support', ascending=False)
supports.head(10)

,support,itemsets
9,0.13,(Banana)
90,0.10,(Organic Strawberries)
8,0.10,(Bag of Organic Bananas)
47,0.07,(Organic Baby Spinach)
35,0.07,(Large Lemon)
68,0.06,(Organic Hass Avocado)
43,0.05,(Organic Avocado)
56,0.05,(Organic Cucumber)
38,0.05,(Limes)
117,0.04,(Strawberries)


In [30]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules

,antecedents,consequents,support,confidence,lift
39,(Organic Red Radish),( Bunch),0.01,1.00,52.17
38,( Bunch),(Organic Red Radish),0.01,0.61,52.17
47,(Organic Baby Spinach),(Organic Garnet Sweet Potato (Yam)),0.01,0.14,6.39
46,(Organic Garnet Sweet Potato (Yam)),(Organic Baby Spinach),0.01,0.48,6.39
51,(Organic Avocado),(Limes),0.01,0.22,4.85
...,...,...,...,...,...
37,(Bag of Organic Bananas),(Large Lemon),0.01,0.12,1.66
3,(Organic Strawberries),(Banana),0.02,0.21,1.64
2,(Banana),(Organic Strawberries),0.02,0.17,1.64
62,(Banana),(Large Lemon),0.01,0.08,1.16
